In [7]:
pip install pandas requests tqdm

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from collections import Counter

# --- 1. CONFIGURAZIONE PERCORSI E PARAMETRI ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

CSV_PATH = os.path.join(PROJECT_ROOT, "data", "raw", "product_dataset.csv")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "data", "images", "products")

MAX_WORKERS = 10

os.makedirs(OUTPUT_DIR, exist_ok=True)

if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(
        f"CSV non trovato in: {CSV_PATH}\n"
        f"Directory corrente: {os.getcwd()}\n"
        f"Assicurati che 'bundle_dataset.csv' sia in '{os.path.join(PROJECT_ROOT, 'data', 'raw')}'"
    )

# --- 2. HEADERS CHE SIMULANO UN BROWSER REALE ---
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
}

# --- 3. FUNZIONE DI DOWNLOAD SINGOLO ---
def download_image(row):
    asset_id = row['bundle_asset_id']
    url = row['bundle_image_url']
    
    save_path = os.path.join(OUTPUT_DIR, f"{asset_id}.jpg")
    
    if os.path.exists(save_path):
        return True, None
        
    try:
        response = requests.get(url, headers=HEADERS, timeout=15)
        response.raise_for_status()
        
        with open(save_path, 'wb') as f:
            f.write(response.content)
        return True, None
    except Exception as e:
        return False, str(e)

# --- 4. ESECUZIONE MULTITHREADING ---
def main():
    print(f"📂 Project root: {PROJECT_ROOT}")
    print(f"Caricamento dataset da {CSV_PATH}...")
    
    df = pd.read_csv(CSV_PATH)
    df = df.dropna(subset=['bundle_image_url'])
    print(f"Trovate {len(df)} immagini da scaricare.")
    
    success_count = 0
    errors = []
    
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(download_image, row): row for _, row in df.iterrows()}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Download in corso"):
            success, error = future.result()
            if success:
                success_count += 1
            elif error:
                errors.append(error)
    
    print(f"\n🚀 Download completato! {success_count}/{len(df)} immagini salvate in: {OUTPUT_DIR}")
    
    # --- 5. RIEPILOGO ERRORI ---
    if errors:
        error_summary = Counter()
        for e in errors:
            if "404" in e:               error_summary["404 Not Found"] += 1
            elif "403" in e:             error_summary["403 Forbidden"] += 1
            elif "429" in e:             error_summary["429 Rate Limited"] += 1
            elif "timed out" in e.lower() or "timeout" in e.lower():
                                         error_summary["Timeout"] += 1
            elif "ConnectionError" in e: error_summary["Connection Error"] += 1
            else:                        error_summary["Altro"] += 1

        print(f"\n📊 Riepilogo errori ({len(errors)} totali):")
        for err_type, count in error_summary.most_common():
            print(f"  {err_type}: {count}")
    else:
        print("\n✅ Nessun errore!")

if __name__ == "__main__":
    main()

📂 Project root: /home/alexbs01/proyectosGit/odsfkfskf
Caricamento dataset da /home/alexbs01/proyectosGit/odsfkfskf/data/raw/product_dataset.csv...


KeyError: ['bundle_image_url']

In [17]:
%pip install torch torchvision torchaudio

Defaulting to user installation because normal site-packages is not writeable
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached setuptools-82.0.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.6.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cufft_cu12-11.3.3.83-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl

In [ ]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install transformers accelerate
!pip install opencv-python Pillow
!pip install faiss-gpu
!pip install rembg
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 32.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 35.1 MB/s eta 0:00:000:00:010:00:0101
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 32.2 MB/s eta 0:00:0031m36.3 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 35.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 30.0 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 37.1 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 36.2 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 35.1 MB/s eta 0:

In [1]:
import torch

print(f"PyTorch version: {torch.version}")
print(f"CUDA disponibile: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  Nessuna GPU — gira su CPU")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n🎯 Device selezionato: {DEVICE}")

PyTorch version: <module 'torch.version' from '/home/aleixbertranandreu/.local/lib/python3.12/site-packages/torch/version.py'>
CUDA disponibile: False
⚠️  Nessuna GPU — gira su CPU

🎯 Device selezionato: cpu


/home/aleixbertranandreu/.local/lib/python3.12/site-packages/torch/cuda/__init__.py:184: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:119.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
import os
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm
import pickle

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🎯 Device: {DEVICE}")

print("Caricamento modello CLIP...")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(DEVICE)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model.eval()
print("✅ Modello CLIP caricato!")

In [ ]:
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
IMAGES_DIR   = os.path.join(PROJECT_ROOT, "data", "images", "products")
OUTPUT_PKL   = os.path.join(PROJECT_ROOT, "data", "embeddings", "catalog_embeddings.pkl")

os.makedirs(os.path.dirname(OUTPUT_PKL), exist_ok=True)

# Lista tutte le immagini disponibili
image_files = [f for f in os.listdir(IMAGES_DIR) if f.endswith(".jpg")]
print(f"📦 Immagini trovate: {len(image_files)}")

# ---- Estrazione embeddings ----
catalog = {}  # { asset_id: embedding_vector }
errors  = []

with torch.no_grad():
    for fname in tqdm(image_files, desc="Estrazione embeddings"):
        asset_id = fname.replace(".jpg", "")
        img_path = os.path.join(IMAGES_DIR, fname)

        try:
            image = Image.open(img_path).convert("RGB")
            inputs = processor(images=image, return_tensors="pt").to(DEVICE)
            embedding = model.get_image_features(**inputs)
            
            # Normalizza il vettore (fondamentale per cosine similarity)
            embedding = embedding / embedding.norm(dim=-1, keepdim=True)
            catalog[asset_id] = embedding.cpu().squeeze().numpy()

        except Exception as e:
            errors.append((asset_id, str(e)))

# Salva il Vector DB
with open(OUTPUT_PKL, "wb") as f:
    pickle.dump(catalog, f)

print(f"\n✅ Vector DB salvato in: {OUTPUT_PKL}")
print(f"   Embeddings estratti: {len(catalog)}")
print(f"   Errori: {len(errors)}")
if errors:
    print("   Primi 5 errori:")
    for asset_id, err in errors[:5]:
        print(f"     - {asset_id}: {err}")